In [ ]:
# Import libraries
import requests
import pandas as pd
import time

In [ ]:
# API Configuration
API_KEY = "579b464db66ec23bdd000001e539e10d69d94c744922ad50125f381e"
RESOURCE_ID = "35985678-0d79-46b4-9ed6-6f13308a1d24"
BASE_URL = "https://api.data.gov.in/resource/"

In [ ]:
# Filters — choose the date you want
DATE_TO_FETCH = "2025-11-07"  # Change this to any date you need
filters = {
    "filters[Arrival_Date]": DATE_TO_FETCH
    # Add more filters if needed, like:
    # "filters[State]": "Maharashtra",
    # "filters[District]": "Pune",
    # "filters[Commodity]": "Tomato"
}

In [ ]:
# Fetch Data with Pagination
def fetch_all_data():
    all_records = []
    offset = 0
    limit = 1000  # Set to max allowed limit
    total_fetched = 0

    while True:
        params = {
            "resource_id": RESOURCE_ID,
            "api-key": API_KEY,
            "format": "json",
            "limit": limit,
            "offset": offset
        }
        params.update(filters)

        print(f"Fetching records {offset + 1} to {offset + limit}...")

        try:
            response = requests.get(BASE_URL + RESOURCE_ID, params=params, timeout=30)
        except Exception as e:
            print(f"⚠️ Request failed at offset {offset}: {e}")
            time.sleep(5)
            continue

        if response.status_code != 200:
            print("⚠️ Failed to fetch data.")
            print("Status Code:", response.status_code)
            print("Response:", response.text)
            break

        data = response.json()
        records = data.get("records", [])

        if not records:
            print("No more records found — all data fetched.")
            break

        all_records.extend(records)
        total_fetched += len(records)

        print(f"Fetched {len(records)} records (Total so far: {total_fetched})")

        # If fewer records than limit — we're done
        if len(records) < limit:
            break

        # Increment offset for next page
        offset += limit
        time.sleep(0.5)  # Small pause to respect API rate limits

    return all_records

In [ ]:

# Main Execution
if __name__ == "__main__":
    print(f"Starting data download for {DATE_TO_FETCH}...\n")
    records = fetch_all_data()

    if records:
        df = pd.DataFrame(records)
        output_file = f"datasets/LiveData[{DATE_TO_FETCH}].csv"
        df.to_csv(output_file, index=False, encoding="utf-8-sig")

        print(f"\n✅ Data saved to '{output_file}' successfully!")
        print(f"Total records saved: {len(df)}")

    else:
        print("⚠️ No data returned for the given filters/date.")

